# Part II | Adding tmdb_api.json

In [1]:
# Load Libraries
import pandas as pd
import numpy as np
import os, json, math, time

from tqdm.notebook import tqdm_notebook

In [2]:
# Double check tmdbsimple Package
#!pip install tmdbsimple

In [3]:
# load compressed csv.gz files
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory=False)
akas = pd.read_csv("Data/title_akas.csv.gz", compression='gzip', low_memory=False)
ratings = pd.read_csv("Data/title.ratings.tsv.gz", compression='gzip', low_memory=False)

In [4]:
# test data heads
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama


In [5]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0


In [6]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1949
1,tt0000002,5.8,264
2,tt0000003,6.5,1778
3,tt0000004,5.6,179
4,tt0000005,6.2,2583


In [7]:
import json
with open('/Users/njfor/.secret/tmdb_api.json') as f:
    login = json.load(f)
## Display the keys of the loaded dictionary
login.keys()

dict_keys(['api-key'])

In [8]:
import tmdbsimple as tmdb
tmdb.API_KEY = login['api-key']

## Querying Movies by ID (Example: The Matrix)

In [9]:
## make a movie object using the .Movies function from tmdb
movie = tmdb.Movies(603)

# 603 = The Matrix

In [10]:
# extract data from movie object
info = movie.info()
info

{'adult': False,
 'backdrop_path': '/waCRuAW5ocONRehP556vPexVXA9.jpg',
 'belongs_to_collection': {'id': 2344,
  'name': 'The Matrix Collection',
  'poster_path': '/bV9qTVHTVf0gkW0j7p7M0ILD4pG.jpg',
  'backdrop_path': '/bRm2DEgUiYciDw3myHuYFInD7la.jpg'},
 'budget': 63000000,
 'genres': [{'id': 28, 'name': 'Action'},
  {'id': 878, 'name': 'Science Fiction'}],
 'homepage': 'http://www.warnerbros.com/matrix',
 'id': 603,
 'imdb_id': 'tt0133093',
 'original_language': 'en',
 'original_title': 'The Matrix',
 'overview': 'Set in the 22nd century, The Matrix tells the story of a computer hacker who joins a group of underground insurgents fighting the vast and powerful computers who now rule the earth.',
 'popularity': 84.071,
 'poster_path': '/f89U3ADr1oiB1s9GkdPOEpXUk5H.jpg',
 'production_companies': [{'id': 79,
   'logo_path': '/tpFpsqbleCzEE2p5EgvUq6ozfCA.png',
   'name': 'Village Roadshow Pictures',
   'origin_country': 'US'},
  {'id': 372,
   'logo_path': None,
   'name': 'Groucho II Film

# Custom Function: get_movie_with_rating 

In [11]:
# custom function to get_movies_with_rating
def get_movie_with_rating(movie_id):
    """Adapted from source = https://github.com/celiao/tmdbsimple"""
    # Get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    
    # save the .info .releases dictionaries
    info = movie.info()
    
    releases = movie.releases()
    # Loop through countires in releases
    for c in releases['countries']:
        # if the country abbreviation==US
        if c['iso_3166_1' ]=='US':
            ## save a "certification" key in info w/ the certification
            info['certification'] = c['certification']
            
    return info

## Testing function (Example: The Avengers)

In [12]:
test = get_movie_with_rating("tt0848228") # put your function name here
test

{'adult': False,
 'backdrop_path': '/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg',
 'belongs_to_collection': {'id': 86311,
  'name': 'The Avengers Collection',
  'poster_path': '/yFSIUVTCvgYrpalUktulvk3Gi5Y.jpg',
  'backdrop_path': '/zuW6fOiusv4X9nnW3paHGfXcSll.jpg'},
 'budget': 220000000,
 'genres': [{'id': 878, 'name': 'Science Fiction'},
  {'id': 28, 'name': 'Action'},
  {'id': 12, 'name': 'Adventure'}],
 'homepage': 'https://www.marvel.com/movies/the-avengers',
 'id': 24428,
 'imdb_id': 'tt0848228',
 'original_language': 'en',
 'original_title': 'The Avengers',
 'overview': 'When an unexpected enemy emerges and threatens global safety and security, Nick Fury, director of the international peacekeeping agency known as S.H.I.E.L.D., finds himself in need of a team to pull the world back from the brink of disaster. Spanning the globe, a daring recruitment effort begins!',
 'popularity': 164.613,
 'poster_path': '/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg',
 'production_companies': [{'id': 420,
   'logo_path

## Saving Our Error Messages

In [13]:
## testing our function by looping through a list of ids
import pandas as pd
test_ids = ["tt0848228", "tt0115937","tt0848228","tt0332280"]
results = []
errors = []
for movie_id in test_ids:
    
    try:
        movie_info = get_movie_with_rating(movie_id)
        results.append(movie_info)
        
    except Exception as e: 
        errors.append([movie_id, e])
    
pd.DataFrame(results)

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,"{'id': 86311, 'name': 'The Avengers Collection...",220000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",https://www.marvel.com/movies/the-avengers,24428,tt0848228,en,The Avengers,...,1518815515,143,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Some assembly required.,The Avengers,False,7.707,28258,PG-13
1,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,"{'id': 86311, 'name': 'The Avengers Collection...",220000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",https://www.marvel.com/movies/the-avengers,24428,tt0848228,en,The Avengers,...,1518815515,143,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Some assembly required.,The Avengers,False,7.707,28258,PG-13
2,False,/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg,None,29000000,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",http://www.newline.com/properties/notebookthe....,11036,tt0332280,en,The Notebook,...,115603229,123,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Behind every great love is a great story.,The Notebook,False,7.878,10183,PG-13


In [14]:
# Check error list
print(f"- Number of errors: {len(errors)}")
errors

- Number of errors: 1


[['tt0115937',
  requests.exceptions.HTTPError('404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0115937?api_key=dacfc8ec089e1a3da2f7c298778f49b1')]]

# BEFORE THE LOOPS (Designate a folder)

In [15]:
# Data Folder
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'title.ratings.tsv.gz',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json']

## Custom Function: write_json

In [16]:
def write_json(new_data, filename):
    """Appends a list of records (new_data) to a json file (filename).
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""
    
    with open(filename, 'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
            file_data.append(new_data)
        # Sets file's current position at offset.
        filel.seek(0)
        # convert back to json.
        json.dump(file_data, file)

# Create Required Lists for the Loop

In [17]:
# Define a list of the Years to Extract from the API
YEARS_TO_GET = [2000,2001]

In [18]:
errors = []

## Create Outer/Inner Loop

### Set Up Progress Bar in Outer/Inner Loop

In [19]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):

    ## Select a JSON_FILE filename to save the results in progress.

    # Define the JSON file to store results for the year
    JSON_FILE=f'{FOLDER}tmdb_api_results_{YEAR}.json'

    # Check if file already exists
    file_exists = os.path.isfile(JSON_FILE)

    # If file doesn't exists: create it
    if file_exists == False:
        # save an empty dict with just "imdb_id" to the new json file
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)

    ## Define/Filter the IDs to call

    # Saving new year as the current df
    df = basics.loc[ basics['startYear']==YEAR].copy()
    # saving movie ids to list
    movie_ids = df['tconst'].copy()

    # Check for & remove any previously downloaded movie id's

    # Load in any existing API resultsl with pd.read_json
    # Check to see if any of the movie_ids to get are already in the JSON_FILE
    # Filter out only movies that are missing from the JSON_FILE to use in the for loop:

    # load existing data from .json into a new dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)

    ## filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

    # Get index & movie_id from list
    # INNER LOOP
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                 desc=f'Movies from {YEAR}',
                                 position=1,
                                 leave=True):
        try:
            # Retrieve the data for the movie_id
            temp = get_movie_with_rating(movie_id)
            # Append/Extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20ms sleep to preven overwhelming the server
            time.sleep(0.02) 

        except Exception as e:
            errors.append([movie_id, e])
        
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1428 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
print(f"- Total errors: {len(errors)}")

# Exploratory Data Analysis
Load in your csv.gz's of results for each year extracted.
- [ ] Concatenate the data into 1 dataframe for the remainder of the analysis.
Once you have your data from the API, they would like you to perform some light EDA to show:
1. How many movies had at least some valid financial information (values > 0 for budget OR revenue)?
Please exclude any movies with 0's for budget AND revenue from the remaining visualizations.
2. How many movies are there in each of the certification categories (G/PG/PG-13/R)?
3. What is the average revenue per certification category?
4. What is the average budget per certification category?

In [ ]:
# Concatenate the data into 1 dataframe
df2000 = pd.read_csv('Data/final_tmdb_data_2000.csv.gz')
df2000